In [ ]:
# Install dependencies when running in Google Colab
!pip install rasterio

In [ ]:
import rasterio
from rasterio.merge import merge
import requests
import os
import tempfile
from osgeo import gdal

## Download Raster Data
To illustrate the visualization of raster, we download [SWISSIMAGE](https://www.swisstopo.admin.ch/en/orthoimage-swissimage-10) data, which is a composition of color aerial photographs over Switzerland with a ground resolution of 10 cm. The data is provided by the Swiss Federal Office of Topography - swisstopo.

[SWISSIMAGE](https://www.swisstopo.admin.ch/en/orthoimage-swissimage-10) stores the orthophotos split into a mosaic of individual files. Here, we download the images covering Zurich and store them in a single GeoTIFF in the local directory `./data/`. The individual download links are listed in [orthophoto_urls.txt](orthophoto_urls.txt).

In [ ]:
url_file_path = "orthophoto_urls.txt"

output_filename = "merged_orthophotos.tif"

data_dir = "./data/"
os.makedirs(data_dir, exist_ok=True)

if not os.path.exists(url_file_path):
    github_url = "https://github.com/geawiz/geoviz/blob/jw/cog_extend/3_Cloud-Optimized-GeoTIFF/orthophoto_urls.txt"
    response = requests.get(github_url)

    with open(url_file_path, 'w') as file:
        file.write(response.text)
  

with open(url_file_path, 'r') as file:
    lines = file.readlines()
    download_urls = [line.strip() for line in lines if line.strip()]

src_files_to_mosaic = []
temp_files = []

try:
    for url in download_urls:
        print(f"Downloading GeoTIFF from: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        geotiff_bytes = response.content

        # Create a temporary file
        temp = tempfile.NamedTemporaryFile(delete=False, suffix='.tif')
        temp.write(geotiff_bytes)
        temp.close()
        temp_files.append(temp.name)
        
        # Open the GeoTIFF with Rasterio
        src = rasterio.open(temp.name)
        src_files_to_mosaic.append(src)
    
    # Merge individual GeoTIFFs
    print("Merging GeoTIFFs")
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    # Update metadata
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    output_path = os.path.join(data_dir, output_filename)
    print(f"Writing merged GeoTIFF to: {output_path}")
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    
    print("Merging completed successfully!")

finally:
    # Cleanup: Close datasets and remove temporary files
    for src in src_files_to_mosaic:
        src.close()
    for temp_path in temp_files:
        os.remove(temp_path)
    print("Temporary files cleaned up.")

In [ ]:
cog_filename = "orthophoto_cog.tif"

target_crs_proj = 'EPSG:3857'

creation_options = [
    'BLOCKSIZE=512',
    'COMPRESS=JPEG2000',
    'NUM_THREADS=ALL_CPUS',
    'BIGTIFF=IF_SAFER',
    'OVERVIEWS=IGNORE_EXISTING'
]

cog_output_path = os.path.join(data_dir, cog_filename)

src_ds = gdal.Open(output_path)

print("Reprojecting and converting to COG...")
dst_ds = gdal.Warp(
    dst_cog_path,
    src_ds,
    dstSRS=target_crs_proj,
    format='COG',
    creationOptions=creation_options,
    multithread=True,
)

 # Flush data to disk
dst_ds = None
src_ds = None